In [83]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import numpy as np

In [93]:
# Carrega modelo e tokenizer
model_name = "neuralmind/bert-base-portuguese-cased"
# model_name = "../trained_model_bert_20250508"
tokenizer_name = model_name # "../tokenizer_model_bert_20250508" 
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [94]:
# Função para gerar embedding médio da sequência
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Média dos embeddings dos tokens (ignorando padding)
    attention_mask = inputs["attention_mask"]
    embeddings = outputs.last_hidden_state
    mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    mean_embedding = sum_embeddings / sum_mask
    return mean_embedding.squeeze().numpy()

In [95]:
# Exemplo: lista de candidatos
candidatos = [
    "Engenheira de ML com foco em PLN e experiência em transformers.",
    "Desenvolvedor Python sênior com interesse em machine learning.",
    "Cientista de dados especializado em análise estatística.",
]

In [96]:
# Gerar os embeddings
candidato_vectors = np.array([get_embedding(c) for c in candidatos]).astype('float32')

In [97]:
candidato_vectors

array([[-0.0026896 , -0.14589028,  0.25189942, ...,  0.33188975,
        -0.0307875 , -0.2937785 ],
       [ 0.24450561, -0.26835427,  0.39407265, ...,  0.3397092 ,
        -0.25220516, -0.48452526],
       [-0.09076482, -0.29330108,  0.9696869 , ..., -0.02338456,
         0.08178648, -0.00464135]], shape=(3, 768), dtype=float32)

In [98]:
candidato_vectors

array([[-0.0026896 , -0.14589028,  0.25189942, ...,  0.33188975,
        -0.0307875 , -0.2937785 ],
       [ 0.24450561, -0.26835427,  0.39407265, ...,  0.3397092 ,
        -0.25220516, -0.48452526],
       [-0.09076482, -0.29330108,  0.9696869 , ..., -0.02338456,
         0.08178648, -0.00464135]], shape=(3, 768), dtype=float32)

In [99]:
# Criar índice FAISS
dimension = candidato_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(candidato_vectors)

In [100]:
# Buscar candidatos mais similares a uma vaga nova
nova_vaga = "Procuramos uma engenheira de machine learning experiente em processamento de linguagem natural e deep learning."
vaga_vector = get_embedding(nova_vaga).astype('float32').reshape(1, -1)

In [101]:
# Buscar top-3 candidatos mais próximos
distances, indices = index.search(vaga_vector, k=3)

# Mostrar resultados
print("Top candidatos similares:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}: {candidatos[idx]} (distância: {distances[0][i]:.4f})")

Top candidatos similares:
1: Desenvolvedor Python sênior com interesse em machine learning. (distância: 16.7690)
2: Engenheira de ML com foco em PLN e experiência em transformers. (distância: 24.8443)
3: Cientista de dados especializado em análise estatística. (distância: 42.1393)


In [ ]:
Top candidatos similares:
1: Engenheira de ML com foco em PLN e experiência em transformers. (distância: 9.3231)
2: Desenvolvedor Python sênior com interesse em machine learning. (distância: 27.1424)
3: Cientista de dados especializado em análise estatística. (distância: 384.6907)

In [ ]:
job_description = "Procuramos uma engenheira de machine learning experiente em processamento de linguagem natural e deep learning."
candidates = [ 
    {"id": 1, "name": "Alice", "resume": "Engenheira de ML com foco em NLP e experiência em transformers."},
    {"id": 2, "name": "Bob", "resume": "Desenvolvedor Python sênior com interesse em machine learning."},
    {"id": 3, "name": "Charlie", "resume": "Cientista de dados especializado em análise estatística."},

# Teste com o Qdrant

In [32]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [30]:
client = QdrantClient(host="localhost", port=6333)

In [60]:
client.create_collection(
    collection_name="applicants",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

client.create_collection(
    collection_name="vacancies",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [61]:
candidatos = [
    {"id": 1, "descricao": "Engenheira de ML com foco em NLP e experiência em transformers."},
    {"id": 2, "descricao": "Desenvolvedor Python sênior com interesse em machine learning."},
    {"id": 3, "descricao": "Cientista de dados especializado em análise estatística."},
]

client.upsert(
    collection_name="applicants",
    points=[
        PointStruct(
            id=c["id"],
            vector=get_embedding(c["descricao"]),
            payload={"descricao": c["descricao"]}
        )
        for c in candidatos
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [64]:
vaga_vector = get_embedding("Estatístico")

# Realiza a busca no índice
resultados = client.query_points(
    collection_name="applicants",
    query=vaga_vector,
    limit=5  # top 5 candidatos mais similares
)

# Mostra os resultados

    #print(f"ID: {r.id} | Score: {r.score:.4f} | Descrição: {r.payload['descricao']}")

In [65]:
for r in resultados.points:
    print(f"ID: {r.id} | Score: {r.score:.4f} | Descrição: {r.payload['descricao']}")

ID: 3 | Score: 0.5543 | Descrição: Cientista de dados especializado em análise estatística.
ID: 1 | Score: 0.3292 | Descrição: Engenheira de ML com foco em NLP e experiência em transformers.
ID: 2 | Score: 0.3207 | Descrição: Desenvolvedor Python sênior com interesse em machine learning.


In [ ]:
scp -r 192.168.101.186:~/Workspace/fiap-2mlet-datathon/data_engineering/tokenizer_model_bert_20250508 ./